In [6]:
import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark
from matplotlib import pyplot as plt
import os
import json
import pickle
from tqdm import tqdm
import subprocess
import random


from qwak_cupy.qwak import QWAK as CQWAK
from qwak.qwak import QWAK as QWAK


In [7]:
def load_list_from_file(file_path):
    with open(file_path, 'r') as file:
        data_str = file.read()
    data = [json.loads(line) for line in data_str.splitlines()]
    return data

def write_list_to_file(file_path, data):
    data_str = [str(item) for item in data]  # Convert float values to strings
    with open(file_path, 'w') as file:
        file.write('\n'.join(data_str))

In [2]:
def runTimedQWAK2(n,pVal,t,seed):
    initNodes = [n//2]
    graph = nx.erdos_renyi_graph(n,pVal,seed=seed)
    start_time = time.time()
    qw = QWAK(graph)
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    final_state = qw.getProbVec()
    return final_state, qwak_time

def runTimedQWAK2_cupy(n,pVal,t,seed):
    start_time = time.time()
    initNodes = [n//2]
    graph = nx.erdos_renyi_graph(n,pVal,seed=seed) 
    qw = CQWAK(graph)
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    final_state = qw.getProbVec()
    return final_state, qwak_time

In [3]:
def runMultipleSimpleQWAK3(nList, pVal, t, samples):#, seed_list_dict):
    qwList = []
    tList = []
    qwak_time = 0
    qwak_time_average = 0

    for n in tqdm(nList, desc=f"NPQWAK {len(nList)}:{nList[0]}->{nList[-1]}", leave=False):
        for sample in tqdm(range(1, samples + 1), desc=f"Samples for N = {n}"):
            qw, qwak_time = runTimedQWAK2(n, pVal, t, 10)
            qwak_time_average += qwak_time

        qwak_time_average = qwak_time_average / samples
        qwList.append(qw)
        tList.append(qwak_time_average)
        qwak_time_average = 0

    return tList, qwList

def runMultipleSimpleQWAK3_cupy(nList, pVal, t, samples):#, seed_list_dict):
    qwList = []
    tList = []
    qwak_time = 0
    qwak_time_average = 0

    for n in tqdm(nList, desc=f"CuPyQWAK {len(nList)}:{nList[0]}->{nList[-1]}", leave=False):
        # Access the corresponding seed list for the current `n`
        for sample in tqdm(range(1, samples + 1), desc=f"Samples for N = {n}"):
            qw, qwak_time = runTimedQWAK2_cupy(n, pVal, t, 10)
            qwak_time_average += qwak_time

        qwak_time_average = qwak_time_average / samples
        qwList.append(qw)
        tList.append(qwak_time_average)
        qwak_time_average = 0

    return tList, qwList

In [4]:
import os
import pickle
from tqdm import tqdm

def runTimedQWAK2(n,pVal,t,seed):
    initNodes = [n//2]
    graph = nx.erdos_renyi_graph(n,pVal,seed=seed)
    start_time = time.time()
    qw = QWAK(graph)
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    final_state = qw.getProbVec()
    return final_state, qwak_time

def runTimedQWAK2_cupy(n,pVal,t,seed):
    start_time = time.time()
    initNodes = [n//2]
    graph = nx.erdos_renyi_graph(n,pVal,seed=seed) 
    qw = CQWAK(graph)
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    final_state = qw.getProbVec()
    return final_state, qwak_time
    
def runMultipleSimpleQWAK3(nList, pVal, t, samples, base_dir):
    qwList = []
    tList = []
    qwak_time_average = 0

    for n in tqdm(nList, desc=f"NPQWAK {len(nList)}:{nList[0]}->{nList[-1]}", leave=False):
        n_dir = os.path.join(base_dir, f"N{n}")
        os.makedirs(n_dir, exist_ok=True)

        t_file = os.path.join(n_dir, f"tList_P{pVal}_T{t}.pkl")
        qw_file = os.path.join(n_dir, f"qwList_P{pVal}_T{t}.pkl")

        if os.path.exists(t_file) and os.path.exists(qw_file):
            print(f"Files for N={n} already exist. Skipping.")
            continue

        tList_n, qwList_n = [], []

        for sample in tqdm(range(1, samples + 1), desc=f"Samples for N = {n}"):
            qw, qwak_time = runTimedQWAK2(n, pVal, t, 10)
            qwak_time_average += qwak_time

            qwList_n.append(qw)
            tList_n.append(qwak_time)

        # Calculate average qwak time for current `n`
        qwak_time_average = sum(tList_n) / len(tList_n)
        tList.append(qwak_time_average)
        qwList.append(qwList_n)

        # Save tList and qwList for current `n`
        with open(t_file, 'wb') as f:
            pickle.dump(tList, f)

        with open(qw_file, 'wb') as f:
            pickle.dump(qwList, f)

    return tList, qwList

def load_runMultipleSimpleQWAK3(nList, pVal, t, base_dir):
    qwList = []
    tList = []

    for n in tqdm(nList, desc="Loading NPQWAK data"):
        n_dir = os.path.join(base_dir, f"N{n}")
        t_file = os.path.join(n_dir, f"tList_P{pVal}_T{t}.pkl")
        qw_file = os.path.join(n_dir, f"qwList_P{pVal}_T{t}.pkl")

        if os.path.exists(t_file) and os.path.exists(qw_file):
            with open(t_file, 'rb') as f:
                tList_n = pickle.load(f)
                tList.append(tList_n)

            with open(qw_file, 'rb') as f:
                qwList_n = pickle.load(f)
                qwList.append(qwList_n)
        else:
            print(f"Files for N={n} not found. Run runMultipleSimpleQWAK3 first.")
            return None, None

    return tList, qwList

def runMultipleSimpleQWAK3_cupy(nList, pVal, t, samples, base_dir):
    qwList = []
    tList = []
    qwak_time_average = 0

    for n in tqdm(nList, desc=f"CuPyQWAK {len(nList)}:{nList[0]}->{nList[-1]}", leave=False):
        n_dir = os.path.join(base_dir, f"N{n}")
        os.makedirs(n_dir, exist_ok=True)

        t_file = os.path.join(n_dir, f"tList_P{pVal}_T{t}_cupy.pkl")
        qw_file = os.path.join(n_dir, f"qwList_P{pVal}_T{t}_cupy.pkl")

        if os.path.exists(t_file) and os.path.exists(qw_file):
            print(f"Files for N={n} already exist. Skipping.")
            continue

        tList_n, qwList_n = [], []

        for sample in tqdm(range(1, samples + 1), desc=f"Samples for N = {n}"):
            qw, qwak_time = runTimedQWAK2_cupy(n, pVal, t, 10)
            qwak_time_average += qwak_time

            qwList_n.append(qw)
            tList_n.append(qwak_time)

        # Calculate average qwak time for current `n`
        qwak_time_average = sum(tList_n) / len(tList_n)
        tList.append(qwak_time_average)
        qwList.append(qwList_n)

        # Save tList and qwList for current `n`
        with open(t_file, 'wb') as f:
            pickle.dump(tList, f)

        with open(qw_file, 'wb') as f:
            pickle.dump(qwList, f)

    return tList, qwList

def load_runMultipleSimpleQWAK3_cupy(nList, pVal, t, base_dir):
    qwList = []
    tList = []

    for n in tqdm(nList, desc="Loading CuPyQWAK data"):
        n_dir = os.path.join(base_dir, f"N{n}")
        t_file = os.path.join(n_dir, f"tList_P{pVal}_T{t}_cupy.pkl")
        qw_file = os.path.join(n_dir, f"qwList_P{pVal}_T{t}_cupy.pkl")

        if os.path.exists(t_file) and os.path.exists(qw_file):
            with open(t_file, 'rb') as f:
                tList_n = pickle.load(f)
                tList.append(tList_n)

            with open(qw_file, 'rb') as f:
                qwList_n = pickle.load(f)
                qwList.append(qwList_n)
        else:
            print(f"Files for N={n} not found. Run runMultipleSimpleQWAK3_cupy first.")
            return None, None

    return tList, qwList


In [12]:
# import os
# from datetime import datetime

# # Parameters
# nMin = 3
# nMax = 500
# nList = list(range(nMin, nMax, 1))
# pVal = 0.8
# samples = 2
# t = 100

# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER'

# # File paths for results
# qwak_times_filename = f'simpleQWAKTime_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
# qwak_times_filename_cupy = f'3070-simpleQWAKTime_CuPy_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'

# qwak_times_file = os.path.join(base_dir, qwak_times_filename)
# qwak_times_file_cupy = os.path.join(base_dir, qwak_times_filename_cupy)

# # Record start datetime
# start_datetime = datetime.now()

# # Run and save results for standard QWAK
# if os.path.exists(qwak_times_file):
#     tList, qwList = load_runMultipleSimpleQWAK3(nList, pVal, t, base_dir)
#     print('Standard QWAK results already exist. Loaded.')
# else:
#     tList, qwList = runMultipleSimpleQWAK3(nList, pVal, t, samples, base_dir)
#     print('Standard QWAK results computed and saved.')

# # Run and save results for CuPy QWAK
# if os.path.exists(qwak_times_file_cupy):
#     tList_cupy, qwList_cupy = load_runMultipleSimpleQWAK3_cupy(nList, pVal, t, base_dir)
#     print('CuPy QWAK results already exist. Loaded.')
# else:
#     tList_cupy, qwList_cupy = runMultipleSimpleQWAK3_cupy(nList, pVal, t, samples, base_dir)
#     print('CuPy QWAK results computed and saved.')

# # Print elapsed time
# elapsed_time = datetime.now() - start_datetime
# print(f"Elapsed time: {elapsed_time}")


Files for N=3 already exist. Skipping.
Files for N=4 already exist. Skipping.
Files for N=5 already exist. Skipping.
Files for N=6 already exist. Skipping.
Files for N=7 already exist. Skipping.
Files for N=8 already exist. Skipping.
Files for N=9 already exist. Skipping.
Files for N=10 already exist. Skipping.
Files for N=11 already exist. Skipping.
Files for N=12 already exist. Skipping.
Files for N=13 already exist. Skipping.
Files for N=14 already exist. Skipping.
Files for N=15 already exist. Skipping.
Files for N=16 already exist. Skipping.
Files for N=17 already exist. Skipping.
Files for N=18 already exist. Skipping.
Files for N=19 already exist. Skipping.
Files for N=20 already exist. Skipping.
Files for N=21 already exist. Skipping.
Files for N=22 already exist. Skipping.
Files for N=23 already exist. Skipping.
Files for N=24 already exist. Skipping.
Files for N=25 already exist. Skipping.
Files for N=26 already exist. Skipping.
Files for N=27 already exist. Skipping.
Files f

CuPyQWAK 497:3->499:   0%|          | 0/497 [00:00<?, ?it/s]

Files for N=3 already exist. Skipping.
Files for N=4 already exist. Skipping.
Files for N=5 already exist. Skipping.
Files for N=6 already exist. Skipping.
Files for N=7 already exist. Skipping.
Files for N=8 already exist. Skipping.
Files for N=9 already exist. Skipping.
Files for N=10 already exist. Skipping.
Files for N=11 already exist. Skipping.
Files for N=12 already exist. Skipping.
Files for N=13 already exist. Skipping.
Files for N=14 already exist. Skipping.
Files for N=15 already exist. Skipping.
Files for N=16 already exist. Skipping.
Files for N=17 already exist. Skipping.
Files for N=18 already exist. Skipping.
Files for N=19 already exist. Skipping.
Files for N=20 already exist. Skipping.
Files for N=21 already exist. Skipping.
Files for N=22 already exist. Skipping.
Files for N=23 already exist. Skipping.
Files for N=24 already exist. Skipping.
Files for N=25 already exist. Skipping.
Files for N=26 already exist. Skipping.
Files for N=27 already exist. Skipping.
Files f

Files for N=362 already exist. Skipping.
Files for N=363 already exist. Skipping.
Files for N=364 already exist. Skipping.
Files for N=365 already exist. Skipping.
Files for N=366 already exist. Skipping.
Files for N=367 already exist. Skipping.
Files for N=368 already exist. Skipping.
Files for N=369 already exist. Skipping.
Files for N=370 already exist. Skipping.
Files for N=371 already exist. Skipping.
Files for N=372 already exist. Skipping.
Files for N=373 already exist. Skipping.
Files for N=374 already exist. Skipping.
Files for N=375 already exist. Skipping.
Files for N=376 already exist. Skipping.
Files for N=377 already exist. Skipping.
Files for N=378 already exist. Skipping.
Files for N=379 already exist. Skipping.
Files for N=380 already exist. Skipping.
Files for N=381 already exist. Skipping.
Files for N=382 already exist. Skipping.
Files for N=383 already exist. Skipping.
Files for N=384 already exist. Skipping.
Files for N=385 already exist. Skipping.
Files for N=386 

In [13]:
import os
import pickle
from tqdm import tqdm

def runMultipleSimpleQWAK3(nList, pVal, t, samples, base_dir):
    qwList = []
    tList = []

    for n in tqdm(nList, desc=f"NPQWAK {len(nList)}:{nList[0]}->{nList[-1]}", leave=False):
        n_dir = os.path.join(base_dir, f"N{n}")
        os.makedirs(n_dir, exist_ok=True)

        for sample in tqdm(range(1, samples + 1), desc=f"Samples for N = {n}", leave=False):
            t_file = os.path.join(n_dir, f"t_P{pVal}_T{t}_sample{sample}.pkl")
            qw_file = os.path.join(n_dir, f"qw_P{pVal}_T{t}_sample{sample}.pkl")

            if os.path.exists(t_file) and os.path.exists(qw_file):
                print(f"Files for N={n}, Sample={sample} already exist. Skipping.")
                continue

            qw, qwak_time = runTimedQWAK2(n, pVal, t, 10)

            # Save t and qw for the current sample
            with open(t_file, 'wb') as f:
                pickle.dump(qwak_time, f)

            with open(qw_file, 'wb') as f:
                pickle.dump(qw, f)

    return

def load_runMultipleSimpleQWAK3(nList, pVal, t, base_dir):
    qwList = []
    tList = []

    for n in tqdm(nList, desc="Loading NPQWAK data"):
        n_dir = os.path.join(base_dir, f"N{n}")

        qwList_n = []
        tList_n = []

        sample = 1
        while True:
            t_file = os.path.join(n_dir, f"t_P{pVal}_T{t}_sample{sample}.pkl")
            qw_file = os.path.join(n_dir, f"qw_P{pVal}_T{t}_sample{sample}.pkl")

            if os.path.exists(t_file) and os.path.exists(qw_file):
                with open(t_file, 'rb') as f:
                    tList_n.append(pickle.load(f))

                with open(qw_file, 'rb') as f:
                    qwList_n.append(pickle.load(f))
            else:
                break

            sample += 1

        if qwList_n and tList_n:
            tList.append(tList_n)
            qwList.append(qwList_n)

    return tList, qwList

def runMultipleSimpleQWAK3_cupy(nList, pVal, t, samples, base_dir):
    qwList = []
    tList = []

    for n in tqdm(nList, desc=f"CuPyQWAK {len(nList)}:{nList[0]}->{nList[-1]}", leave=False):
        n_dir = os.path.join(base_dir, f"N{n}")
        os.makedirs(n_dir, exist_ok=True)

        for sample in tqdm(range(1, samples + 1), desc=f"Samples for N = {n}", leave=False):
            t_file = os.path.join(n_dir, f"t_P{pVal}_T{t}_sample{sample}_cupy.pkl")
            qw_file = os.path.join(n_dir, f"qw_P{pVal}_T{t}_sample{sample}_cupy.pkl")

            if os.path.exists(t_file) and os.path.exists(qw_file):
                print(f"Files for N={n}, Sample={sample} already exist. Skipping.")
                continue

            qw, qwak_time = runTimedQWAK2_cupy(n, pVal, t, 10)

            # Save t and qw for the current sample
            with open(t_file, 'wb') as f:
                pickle.dump(qwak_time, f)

            with open(qw_file, 'wb') as f:
                pickle.dump(qw, f)

    return

def load_runMultipleSimpleQWAK3_cupy(nList, pVal, t, base_dir):
    qwList = []
    tList = []

    for n in tqdm(nList, desc="Loading CuPyQWAK data"):
        n_dir = os.path.join(base_dir, f"N{n}")

        qwList_n = []
        tList_n = []

        sample = 1
        while True:
            t_file = os.path.join(n_dir, f"t_P{pVal}_T{t}_sample{sample}_cupy.pkl")
            qw_file = os.path.join(n_dir, f"qw_P{pVal}_T{t}_sample{sample}_cupy.pkl")

            if os.path.exists(t_file) and os.path.exists(qw_file):
                with open(t_file, 'rb') as f:
                    tList_n.append(pickle.load(f))

                with open(qw_file, 'rb') as f:
                    qwList_n.append(pickle.load(f))
            else:
                break

            sample += 1

        if qwList_n and tList_n:
            tList.append(tList_n)
            qwList.append(qwList_n)

    return tList, qwList
    
import os
from datetime import datetime

# Parameters
nMin = 3
nMax = 500
nList = list(range(nMin, nMax, 1))
pVal = 0.8
samples = 3
t = 100

base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER'

# File paths for results
qwak_times_filename = f'simpleQWAKTime_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
qwak_times_filename_cupy = f'3070-simpleQWAKTime_CuPy_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'

qwak_times_file = os.path.join(base_dir, qwak_times_filename)
qwak_times_file_cupy = os.path.join(base_dir, qwak_times_filename_cupy)

# Record start datetime
start_datetime = datetime.now()

# Run and save results for standard QWAK
if os.path.exists(qwak_times_file):
    tList, qwList = load_runMultipleSimpleQWAK3(nList, pVal, t, base_dir)
    print('Standard QWAK results already exist. Loaded.')
else:
    tList, qwList = runMultipleSimpleQWAK3(nList, pVal, t, samples, base_dir)
    print('Standard QWAK results computed and saved.')

# Run and save results for CuPy QWAK
if os.path.exists(qwak_times_file_cupy):
    tList_cupy, qwList_cupy = load_runMultipleSimpleQWAK3_cupy(nList, pVal, t, base_dir)
    print('CuPy QWAK results already exist. Loaded.')
else:
    tList_cupy, qwList_cupy = runMultipleSimpleQWAK3_cupy(nList, pVal, t, samples, base_dir)
    print('CuPy QWAK results computed and saved.')

# Print elapsed time
elapsed_time = datetime.now() - start_datetime
print(f"Elapsed time: {elapsed_time}")


Samples for N = 3:   0%|          | 0/3 [00:00<?, ?it/s]
                                                        

Files for N=3, Sample=1 already exist. Skipping.
Files for N=3, Sample=2 already exist. Skipping.
Files for N=3, Sample=3 already exist. Skipping.



Samples for N = 4:   0%|          | 0/3 [00:00<?, ?it/s]
                                                        

Files for N=4, Sample=1 already exist. Skipping.
Files for N=4, Sample=2 already exist. Skipping.
Files for N=4, Sample=3 already exist. Skipping.



Samples for N = 5:   0%|          | 0/3 [00:00<?, ?it/s]
                                                        

...


Samples for N = 425:   0%|          | 0/3 [00:00<?, ?it/s]
                                                          

Files for N=425, Sample=1 already exist. Skipping.
Files for N=425, Sample=2 already exist. Skipping.
Files for N=425, Sample=3 already exist. Skipping.



Samples for N = 426:   0%|          | 0/3 [00:00<?, ?it/s]
                                                          

Files for N=426, Sample=1 already exist. Skipping.
Files for N=426, Sample=2 already exist. Skipping.
Files for N=426, Sample=3 already exist. Skipping.



Samples for N = 427:   0%|          | 0/3 [00:00<?, ?it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



NameError: name 'qwList' is not defined